# Evaluating a Classifier based on deep learning

Codes in this notebook are executed on a Linux-based virtual machine with the following **computational requirements**:
* GPU:  RTX2080 Super
* vCPU:  8 
* CPU Memory: 48GB 
* GPU Memory: 8GB

## Import necessary dependencies and data

In [1]:
import os
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import tensorflow as tf

2025-03-07 00:16:03.874928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741324563.901833  271035 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741324563.910017  271035 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-07 00:16:03.941618: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Load the Model
model_path = 'models/model_deep_learning_distilBERT'

if os.path.exists(model_path):
    if os.path.isdir(model_path):
        files = os.listdir(model_path)
        for file in files:
            if file.endswith('.h5'):
                print("Model file exists inside the directory.")
                try:
                    model = tf.keras.models.load_model(os.path.join(model_path, file))
                    model.summary()
                except ValueError as e:
                    print(f"Error loading model: {e}")
                break
        else:
            print("Model file does not exist inside the directory.")
    else:
        print("Model path is a file.")
else:
    print("Model path does not exist.")

Model file exists inside the directory.
Error loading model: No model config found in the file at models/model_deep_learning_distilBERT/tf_model.h5.
